# Demonstrates how to load input CSV files and run them through PALS

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pathlib
import pickle

sys.path.append('..')

In [3]:
import pandas as pd

In [4]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.feature_extraction import DataSource
from pals.PLAGE import PLAGE
from pals.ORA import ORA
from pals.GSEA import GSEA
from pals.common import *

2020-10-13 16:05:38.293 | WARNING  | pals.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2020-10-13 16:05:38.294 | WARNING  | pals.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2020-10-13 16:05:38.297 | WARNING  | pals.reactome:get_neo4j_driver:27 - Failed to connect to graph database: Failed to establish connection to IPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused)
2020-10-13 16:05:38.298 | WARNING  | pals.reactome:<module>:34 - Driver initialisation failed. PALS will run without Reactome support.


# Beer Analysis

### Load data

In [23]:
osp = '/Volumes/Transcend2/17_20_PhD/19_20_PhD_Metabolomics/'
intensity_csv = '/Users/anamaria/Desktop/samples_filtered.csv'
annotation_csv ='/Users/anamaria/Desktop/annotation.csv'
sd = pd.read_csv(osp+'pymz/mzmine/peak_picked_files/sample_description.csv')

experimental_design =  {
    'comparisons': [{'case': 'infected', 'control': 'control', 'name':'infected/control'}],
    'groups': {'infected': [], 'control': []}
}    

for case in experimental_design['groups']:
    if case == 'infected':
        experimental_design['groups'][case] = [s for s in sd[sd['Condition'] == 'infected']['Sample Name']]
    if case == 'control':
        experimental_design['groups'][case] = [s for s in sd[sd['Condition'] == 'control']['Sample Name']]

In [16]:
int_df, annotation_df, groups = load_data(intensity_csv, annotation_csv)

2020-10-13 16:09:39.698 | DEBUG    | pals.common:load_data:165 - Loaded 603 x 76 peak intensities from /Users/anamaria/Desktop/samples_filtered.csv
2020-10-13 16:09:39.700 | DEBUG    | pals.common:load_data:166 - Loaded groups: {'1': ['row ID'], '28621253.99112652': ['ZIK6.mzXML'], '24640484.38107913': ['ZIK7.mzXML'], '33008265.78013173': ['ZIK10.mzXML'], '42847335.83581253': ['C7_2.mzXML'], '17186613.05137111': ['KM_10.mzXML'], '20854256.241177734': ['C1.mzXML'], '48234259.52069932': ['C10_2.mzXML'], '28440550.373382807': ['VL7.mzXML'], '13797797.795971667': ['KM_9.mzXML'], '14035107.721769508': ['KM_11.mzXML'], '18056019.41098433': ['KM_8.mzXML'], '32745048.04885156': ['KM_3.mzXML'], '45397271.78259371': ['C18_2.mzXML'], '48322070.51710834': ['KM_2.mzXML'], '43252999.04044532': ['C2_2.mzXML'], '32269382.197101586': ['C5_2.mzXML'], '36125496.58238283': ['C12_2.mzXML'], '83811408.14835934': ['VL17.mzXML'], '59913060.78692186': ['VL16.mzXML'], '25800199.16220113': ['KM_4.mzXML'], '21102

In [17]:
int_df.head()

,ZIK6.mzXML,ZIK7.mzXML,ZIK10.mzXML,C7_2.mzXML,KM_10.mzXML,C1.mzXML,C10_2.mzXML,VL7.mzXML,KM_9.mzXML,KM_11.mzXML,...,KM_7.mzXML,KM_6.mzXML,C4_2.mzXML,C13_2.mzXML,C10.mzXML,C9_2.mzXML,VL13.mzXML,VL12.mzXML,VL19.mzXML,VL18.mzXML
row ID,,,,,,,,,,,,,,,,,,,,,
2051,6.085213e+07,6.872525e+07,7.030129e+07,1.746869e+08,2.163887e+07,7.888700e+07,3.125270e+08,5.353562e+07,2.537424e+07,2.325119e+07,...,2.939156e+07,2.248375e+07,1.003125e+08,2.069511e+08,6.199823e+07,1.192371e+08,2.546797e+07,1.311025e+08,8.261907e+07,0.000000e+00
6,2.517629e+07,2.658106e+07,3.733972e+07,6.462868e+07,1.607754e+07,3.058282e+07,4.819649e+07,3.886415e+07,1.667960e+07,1.653038e+07,...,1.322915e+07,1.862137e+07,4.926671e+07,4.255550e+07,3.691508e+07,5.726292e+07,4.069165e+07,6.084040e+07,5.982381e+07,3.035807e+07
2057,3.929179e+06,4.057528e+06,6.033983e+06,1.775027e+07,1.254025e+06,1.036919e+07,5.960417e+06,8.272370e+06,0.000000e+00,2.273807e+06,...,1.007962e+06,1.179840e+06,8.422987e+06,6.063877e+06,1.709164e+07,1.639748e+07,9.545597e+05,5.113669e+07,2.735610e+07,0.000000e+00
10,4.342657e+06,2.640288e+06,3.978803e+06,6.722572e+06,2.854626e+06,3.275266e+06,5.625803e+06,4.703867e+06,0.000000e+00,2.818107e+06,...,2.552461e+06,2.267662e+06,8.397378e+06,7.129759e+06,5.844209e+06,6.561646e+06,4.430804e+06,9.189017e+06,7.417835e+06,3.923343e+06
2058,7.744800e+06,7.566545e+06,9.511789e+06,2.245973e+07,1.386831e+06,9.665703e+06,3.617053e+07,6.607266e+06,2.555483e+06,9.970690e+05,...,2.304320e+06,2.400551e+06,1.256284e+07,2.360607e+07,7.232901e+06,1.378934e+07,2.386602e+06,1.605226e+07,1.172011e+07,0.000000e+00


In [18]:
annotation_df.head()

,entity_id
peak_id,
633,C00418
613,C00064
3109,NaN
2989,NaN
324,C00245


Define some comparisons. This should be specified by users from the interface.

For simplicity, we can just let user specify one comparison at a time (what is currently the case in PALS Viewer), although the codes allow us to specify multiple comparisons.

In [16]:
comparisons = [
    ('beer1', 'beer2'), 
    # ('beer3', 'beer4')
]

Create experimental design dictionary

In [19]:
experimental_design = {
    'groups': groups,
    'comparisons': []
}
for case, control in comparisons:
    experimental_design['comparisons'].append({
        'case': case,
        'control': control,
        'name': '%s/%s' % (case, control)
    })
experimental_design

{'groups': {'beer1': ['Beer_1_full1.mzXML',
   'Beer_1_full2.mzXML',
   'Beer_1_full3.mzXML'],
  'beer2': ['Beer_2_full1.mzXML', 'Beer_2_full2.mzXML', 'Beer_2_full3.mzXML'],
  'beer3': ['Beer_3_full1.mzXML', 'Beer_3_full2.mzXML', 'Beer_3_full3.mzXML'],
  'beer4': ['Beer_4_full1.mzXML', 'Beer_4_full2.mzXML', 'Beer_4_full3.mzXML']},
 'comparisons': [{'case': 'beer1', 'control': 'beer2', 'name': 'beer1/beer2'},
  {'case': 'beer3', 'control': 'beer4', 'name': 'beer3/beer4'}]}

In [24]:
experimental_design

{'comparisons': [{'case': 'infected',
   'control': 'control',
   'name': 'infected/control'}],
 'groups': {'infected': ['ZIK6.mzXML',
   'ZIK7.mzXML',
   'ZIK10.mzXML',
   'VL7.mzXML',
   'KM_9.mzXML',
   'KM_11.mzXML',
   'KM_8.mzXML',
   'KM_3.mzXML',
   'KM_2.mzXML',
   'VL17.mzXML',
   'VL16.mzXML',
   'KM_5.mzXML',
   'VL10.mzXML',
   'VL1.mzXML',
   'VL11.mzXML',
   'ZIK1.mzXML',
   'KM_17.mzXML',
   'VL15.mzXML',
   'VL14.mzXML',
   'KM_12.mzXML',
   'KM_18.mzXML',
   'ZIK4.mzXML',
   'ZIK5.mzXML',
   'VL4.mzXML',
   'VL5.mzXML',
   'ZIK8.mzXML',
   'ZIK9.mzXML',
   'KM_14.mzXML',
   'KM_15.mzXML',
   'VL8.mzXML',
   'VL9.mzXML',
   'KM_20.mzXML',
   'KM_21.mzXML',
   'ZIK3.mzXML',
   'VL3.mzXML',
   'ZIK2.mzXML',
   'KM_6.mzXML',
   'VL13.mzXML',
   'VL12.mzXML',
   'VL19.mzXML',
   'VL18.mzXML'],
  'control': ['C7_2.mzXML',
   'KM_10.mzXML',
   'C1.mzXML',
   'C10_2.mzXML',
   'C18_2.mzXML',
   'C2_2.mzXML',
   'C5_2.mzXML',
   'C12_2.mzXML',
   'KM_4.mzXML',
   'C8_2.mzXML',

### PALS analysis using KEGG database exported from PiMP

In [25]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_PIMP_KEGG)

2020-10-13 16:13:53.777 | DEBUG    | pals.feature_extraction:__init__:43 - Using PiMP_KEGG as database
2020-10-13 16:13:53.779 | DEBUG    | pals.loader:load_data:42 - Loading /Users/anamaria/git/PALS/pals/data/PiMP_KEGG.json.zip
2020-10-13 16:13:53.980 | DEBUG    | pals.feature_extraction:__init__:56 - Mapping pathway to unique ids
2020-10-13 16:13:53.989 | DEBUG    | pals.feature_extraction:__init__:70 - Creating dataset to pathway mapping
2020-10-13 16:13:54.131 | DEBUG    | pals.feature_extraction:__init__:98 - Computing unique id counts


In [26]:
plage = PLAGE(ds)
pathway_df = plage.get_pathway_df()

2020-10-13 16:13:56.616 | DEBUG    | pals.feature_extraction:change_zero_peak_ints:308 - Setting the zero intensity values in the dataframe
2020-10-13 16:13:56.625 | DEBUG    | pals.feature_extraction:change_zero_peak_ints:310 - 0
2020-10-13 16:13:56.691 | DEBUG    | pals.feature_extraction:standardize_intensity_df:277 - Scaling the data across the sample: zero mean and unit variance
2020-10-13 16:13:56.723 | DEBUG    | pals.PLAGE:get_plage_activity_df:90 - Mean values of the rows in the DF is [ 0.  0. -0. -0. -0. -0. -0.  0.  0. -0. -0.  0. -0.  0. -0. -0. -0.  0.
  0.  0.  0.  0. -0.  0.  0. -0.  0.  0. -0.  0. -0. -0. -0. -0.  0.  0.
 -0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0.  0. -0. -0.  0.  0.
 -0.  0.  0. -0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0. -0.  0.  0.
  0. -0. -0. -0.  0.  0.  0. -0. -0. -0. -0. -0. -0. -0. -0.  0. -0.  0.
 -0.  0. -0. -0.  0.  0. -0.  0.  0. -0.  0. -0.  0. -0.  0. -0. -0. -0.
  0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0.  0.  0.  0.  0

In [30]:
pd.set_option('display.max_rows', 500)
pathway_df.sort_values('PiMP_KEGG infected/control comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,infected/control p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,PiMP_KEGG infected/control comb_p
map00750,Vitamin B6 metabolism,0.000000e+00,29,5,17.24,2.176285e-04,0.58,2.0,0.000000e+00
map00240,Pyrimidine metabolism,4.342559e-09,56,5,8.93,4.643343e-03,1.12,2.0,4.342559e-09
map00471,D-Glutamine and D-glutamate metabolism,2.582631e-08,7,2,28.57,7.697120e-03,0.14,2.0,2.582631e-08
map04964,Proximal tubule bicarbonate reclamation,2.582631e-08,16,2,12.50,3.921214e-02,0.32,2.0,2.582631e-08
map04724,Glutamatergic synapse,2.582631e-08,7,2,28.57,7.697120e-03,0.14,2.0,2.582631e-08
map04727,GABAergic synapse,2.582631e-08,9,2,22.22,1.286096e-02,0.18,2.0,2.582631e-08
map00910,Nitrogen metabolism,2.582631e-08,17,2,11.76,4.388109e-02,0.34,2.0,2.582631e-08
map04020,Calcium signaling pathway,1.261071e-02,9,1,11.11,1.660377e-01,0.18,2.0,1.261071e-02
map05152,Tuberculosis,1.261071e-02,6,1,16.67,1.139487e-01,0.12,2.0,1.261071e-02
map04666,Fc gamma R-mediated phagocytosis,1.261071e-02,4,1,25.00,7.746050e-02,0.08,2.0,1.261071e-02
